## Part 1

The goal here is to ensure that we use the lines inside the input.txt in such a way that we always pair the closest pairs to one another in circuits.
This will be used in conjunction with Kruskal's algorithm or something similar to ensure that every point is connected to at least a single other point. Kruskal showed most promise in comparison to Prim's algorithm. Prim's does not focus on multiple sub-trees which is what we want to track here.

In [4]:
from pathlib import Path

# Read input
input_path = Path('input.txt')
points = []
with open(input_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            coords = list(map(int, line.split(',')))
            points.append(coords)

n = len(points)

# Calculate all pairwise distances
def squared_euclidean_distance(p1, p2):
    return sum((a - b) ** 2 for a, b in zip(p1, p2))

# Create all edges with distances
edges = []
for i in range(n):
    for j in range(i + 1, n):
        dist = squared_euclidean_distance(points[i], points[j])
        edges.append((dist, i, j))

# Sort edges by distance
edges.sort()

# Union-Find data structure
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.size = [1] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)

        if root_x == root_y:
            return False

        # Union by size
        if self.size[root_x] < self.size[root_y]:
            self.parent[root_x] = root_y
            self.size[root_y] += self.size[root_x]
        else:
            self.parent[root_y] = root_x
            self.size[root_x] += self.size[root_y]
        return True

    def get_size(self, x):
        root = self.find(x)
        return self.size[root]

    def all_nodes_in_tree(self):
        """Check if every node is in a tree of size >= 2"""
        for node in range(len(self.parent)):
            if self.get_size(node) < 2:
                return False
        return True

uf = UnionFind(n)
edges_processed = 0
connections_made = 0
# In this case, it is number of connections ATTEMPTED
NUM_CONNECTIONS_ATTEMPTED = NUM_CONNECTIONS_NEEDED = 1000

for dist, u, v in edges:
    edges_processed += 1
    uf.union(u, v)
    if edges_processed % NUM_CONNECTIONS_ATTEMPTED == 0:
        break


# Get all unique tree sizes
tree_sizes = {}
for node in range(n):
    root = uf.find(node)
    if root not in tree_sizes:
        tree_sizes[root] = uf.get_size(root)

# Sort tree sizes in descending order
list_of_sizes = sorted(tree_sizes.values(), reverse=True)
if len(list_of_sizes) >= 3:
    result = list_of_sizes[0] * list_of_sizes[1] * list_of_sizes[2]
    print(f"Top 3 circuit sizes multiplied: {result}")


Top 3 circuit sizes multiplied: 32103


## Part 2

The goal is now to basically check the final edge that connects the final two constellations together. This should be relatively simple by simply checking the last edge that was added to the union-find structure.

In [6]:
final_uf = UnionFind(n)

connections_made = 0
final_pair = None
for dist, u, v in edges:
    if final_uf.union(u, v):
        connections_made += 1
        if connections_made == NUM_CONNECTIONS_NEEDED - 1:
            final_pair = (u, v)
            break

final_answer = points[final_pair[0]][0] * points[final_pair[1]][0]

print("Final pair of X points multiplied: ", final_answer)

Final pair of X points multiplied:  8133642976
